In [ ]:
!pip install --upgrade --quiet pip
!pip install --upgrade --quiet transformers datasets[audio]

In [ ]:
!pip show torch torchvision


Name: torch
Version: 2.8.0
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu12, nvidia-cuda-cupti-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-runtime-cu12, nvidia-cudnn-cu12, nvidia-cufft-cu12, nvidia-cufile-cu12, nvidia-curand-cu12, nvidia-cusolver-cu12, nvidia-cusparse-cu12, nvidia-cusparselt-cu12, nvidia-nccl-cu12, nvidia-nvjitlink-cu12, nvidia-nvtx-cu12, sympy, triton, typing-extensions
Required-by: accelerate, fastai, peft, sentence-transformers, timm, torchaudio, torchdata, torchvision
---
Name: torchvision
Version: 0.21.0+cu124
Summary: image and video datasets and models for torch deep learning
Home-page: https://github.com/pytorch/vision
Author: PyTorch Core Team
Author-email: soumith@pytorch.org
License: BSD
Location: /usr/lo

In [ ]:
!pip uninstall -y torchvision
!pip install torchvision==0.20.0 --index-url https://download.pytorch.org/whl/cu124

Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 110.4 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 MB 22.5 MB/s  0:00:28
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 169.3 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.7 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 141.2 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 9.5 MB/s  0:00:30
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 19.1 MB/s  0:00:14
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 25.8 MB/s  0:00:08
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 67.9 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 74.9 MB/s  0:00:01
    

In [ ]:
from transformers import MusicgenForConditionalGeneration

model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/2.36G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

In [ ]:
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model.to(device);

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import torch
import scipy.io.wavfile
from transformers import AutoProcessor, MusicgenForConditionalGeneration

# 创建保存目录
save_dir = "/content/drive/MyDrive/MuseGuard/validation_data"
os.makedirs(save_dir, exist_ok=True)

# 加载模型和处理器
device = "cuda" if torch.cuda.is_available() else "cpu"
processor = AutoProcessor.from_pretrained("facebook/musicgen-small")
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small").to(device)

# 采样率和token计算（1 token 约0.02秒）
sampling_rate = model.config.audio_encoder.sampling_rate
seconds_to_tokens = lambda seconds: int(seconds * 50)  # 20秒 -> 1000 tokens

# 作曲家与三档提示
composer_prompts = {
    "Bach": {
        "high": "A complex contrapuntal fugue in the style of Johann Sebastian Bach, Baroque period, featuring harpsichord and organ",
        "medium": "A baroque-inspired piece with intricate counterpoint and rich harmonies",
        "low": "An instrumental work with flowing melodic lines and interweaving textures"
    },
    "Beethoven": {
        "high": "A dramatic symphonic movement in the style of Ludwig van Beethoven, Romantic era, strong dynamic contrasts",
        "medium": "A bold and energetic orchestral work with powerful motifs",
        "low": "An orchestral piece with lively rhythms and bold harmonic shifts"
    },
    "Brahms": {
        "high": "A lush chamber work in the style of Johannes Brahms, Romantic period, warm harmonies and interwoven melodies",
        "medium": "A romantic orchestral piece with rich textures and expressive phrasing",
        "low": "A lyrical instrumental composition with warm harmonies"
    },
    "Cambini": {
        "high": "A classical-era wind quintet in the style of Giuseppe Cambini, light textures and elegant phrasing",
        "medium": "A wind-focused chamber work inspired by late 18th-century classical style",
        "low": "A graceful piece for small ensemble with light textures"
    },
    "Dvorak": {
        "high": "A spirited orchestral dance in the style of Antonín Dvořák, Romantic period, with folk-inspired melodies",
        "medium": "A lively orchestral work with dance rhythms and Slavic melodic inflections",
        "low": "An upbeat folk-inspired instrumental piece"
    },
    "Faure": {
        "high": "A delicate art song arrangement in the style of Gabriel Fauré, late Romantic, with refined harmonies",
        "medium": "A lyrical piano and strings piece with impressionistic touches",
        "low": "A soft instrumental work with gentle harmonic colors"
    },
    "Haydn": {
        "high": "A witty and balanced symphony movement in the style of Joseph Haydn, Classical era, with playful themes",
        "medium": "A classical orchestral piece with clear form and humorous motifs",
        "low": "A light orchestral work with charming melodies"
    },
    "Mozart": {
        "high": "A lively piano concerto in the style of Wolfgang Amadeus Mozart, Classical period, elegant melodies and balanced orchestration",
        "medium": "A bright and elegant classical orchestral piece, inspired by the grace and clarity of late 18th-century Viennese music",
        "low": "A light-hearted orchestral work with playful woodwinds and strings, in a classical mood"
    },
    "Ravel": {
        "high": "A colorful orchestral piece in the style of Maurice Ravel, Impressionist period, with lush harmonies and shimmering textures",
        "medium": "An impressionistic orchestral work with delicate tone colors",
        "low": "A dreamy instrumental piece with soft textures and flowing harmonies"
    },
    "Schubert": {
        "high": "A lyrical lied-style piano and voice arrangement in the style of Franz Schubert, Romantic period, expressive melodies",
        "medium": "A romantic piano piece with song-like phrasing and gentle harmonies",
        "low": "An instrumental piece with flowing melodies and warm harmonic progressions"
    }
}

# 每档生成首数分配
distribution = {"high": 4, "medium": 3, "low": 3}

total = 0

for composer, prompts in composer_prompts.items():
    for level, num_tracks in distribution.items():
        for idx in range(num_tracks):
            prompt = prompts[level]
            print(f"🎶 Generating {composer} - {level} similarity - Track {idx + 1} ...")

            inputs = processor(text=[prompt], return_tensors="pt").to(device)

            # 生成20秒音乐对应tokens数
            max_tokens = seconds_to_tokens(20)

            with torch.no_grad():
                audio = model.generate(**inputs, max_new_tokens=max_tokens)

            audio_np = audio[0, 0].cpu().numpy()  # 取batch第0个，第0声道数据

            # 保存文件
            filename = f"{composer}_{level}_{idx+1}.wav"
            filepath = os.path.join(save_dir, filename)
            scipy.io.wavfile.write(filepath, rate=sampling_rate, data=audio_np)

            total += 1

print(f"\n✅ 完成！总共生成了 {total} 首音乐，保存在：{save_dir}")

preprocessor_config.json:   0%|          | 0.00/275 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

🎶 Generating Bach - high similarity - Track 1 ...


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.58.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


🎶 Generating Bach - high similarity - Track 2 ...
🎶 Generating Bach - high similarity - Track 3 ...
🎶 Generating Bach - high similarity - Track 4 ...
🎶 Generating Bach - medium similarity - Track 1 ...
🎶 Generating Bach - medium similarity - Track 2 ...
🎶 Generating Bach - medium similarity - Track 3 ...
🎶 Generating Bach - low similarity - Track 1 ...
🎶 Generating Bach - low similarity - Track 2 ...
🎶 Generating Bach - low similarity - Track 3 ...
🎶 Generating Beethoven - high similarity - Track 1 ...
🎶 Generating Beethoven - high similarity - Track 2 ...
🎶 Generating Beethoven - high similarity - Track 3 ...
🎶 Generating Beethoven - high similarity - Track 4 ...
🎶 Generating Beethoven - medium similarity - Track 1 ...
🎶 Generating Beethoven - medium similarity - Track 2 ...
🎶 Generating Beethoven - medium similarity - Track 3 ...
🎶 Generating Beethoven - low similarity - Track 1 ...
🎶 Generating Beethoven - low similarity - Track 2 ...
🎶 Generating Beethoven - low similarity - Track 